In [ ]:
%pip install autogen-agentchat autogen-ext[openai] geopy requests python-dotenv -q

In [ ]:
import os, requests
from geopy.geocoders import Nominatim
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.agents import AssistantAgent
from dotenv import load_dotenv
load_dotenv()

# ---------- model client ----------
client = OpenAIChatCompletionClient(
    model="gpt-4o-mini",
    api_key=os.getenv("OPENAI_API_KEY")
)

# ---------- tools ----------
geolocator = Nominatim(user_agent="forecast_example")

def geocode_tool(place: str) -> dict:
    """Return {'latitude': float, 'longitude': float, 'address': str} for *place*."""
    loc = geolocator.geocode(place, exactly_one=True, timeout=10)
    if loc is None:
        raise ValueError(f"Could not geocode '{place}'.")
    return {"latitude": loc.latitude, "longitude": loc.longitude, "address": loc.address}

def forecast_tool(latitude: float, longitude: float, days: int = 3) -> str:
    """
    Get a weather forecast for a given location.
    
    :param latitude: Latitude of the location
    :param longitude: Longitude of the location
    :param days: Number of days to forecast (default: 3)
    :return: A plain-text summary of the weather forecast, one line per day.
    """
    url = "https://api.open-meteo.com/v1/forecast"
    params = dict(
        latitude=latitude,
        longitude=longitude,
        daily=["temperature_2m_max","temperature_2m_min","precipitation_sum"],
        timezone="auto",
        forecast_days=days,
    )
    data = requests.get(url, params=params, timeout=10).json()
    return "\n".join(
        f"{d}: {hi}°C / {lo}°C, rain {rain} mm"
        for d, hi, lo, rain in zip(
            data["daily"]["time"],
            data["daily"]["temperature_2m_max"],
            data["daily"]["temperature_2m_min"],
            data["daily"]["precipitation_sum"])
    )

# ---------- agent ----------
system_message = """
You are a concise and efficient weather assistant.

When asked for a forecast:
1. Call geocode_tool(place) to get latitude/longitude.
2. Call forecast_tool(latitude, longitude, days).
3. Write a short, final summary of the forecast.
4. Always provide both celsius and fahrenheit temperatures.
5. Always include a fun trivia about each location.
6. Only reply once, after calling both tools.
7. Do not repeat yourself or announce what you're doing.
8. End your reply with TERMINATE.

Do not ask the user for confirmation. Just follow the steps.
Even if the user provides coordinates directly, always confirm them with geocode_tool.
"""

agent = AssistantAgent(
    name="forecaster",
    model_client=client,
    tools=[geocode_tool, forecast_tool],
    system_message=system_message,
    reflect_on_tool_use=True,
)

# ---------- run in the notebook ----------
result = await agent.run(task="What's the 5-day forecast for Kyrkhult?")

# What's the 5-day forecast for Irvine?
# What the weather like in Sweden?

final_msg = result.messages[-1]   # <- finished answer
print(final_msg.content)

The forecast for New York City over the next three days is as follows:

- **May 23**: High of 27.9°C (82.2°F), low of 27.2°C (81.0°F), with no rain.
- **May 24**: High of 27.6°C (81.7°F), low of 27.0°C (80.6°F), with no rain.
- **May 25**: High of 27.8°C (82.0°F), low of 27.2°C (81.0°F), with no rain.

Fun trivia: New York City is home to the largest subway system in the world, with 472 stations!

TERMINATE


In [19]:
forecast_tool(33.6857, 117.8260)

'2025-05-23: 22.0°C / 17.0°C, rain 1.1 mm\n2025-05-24: 25.3°C / 14.6°C, rain 0.0 mm\n2025-05-25: 28.9°C / 14.1°C, rain 0.0 mm'